In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Embedding, GRU, Bidirectional
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import re
import json
import joblib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import random

# ----------------------------------------
# Set the output directory
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/GRU_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ----------------------------------------
# Define the text preprocessing function
def preprocess_text(text):
    # Convert to lowercase and remove commas and periods
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# ----------------------------------------
# Read the data
data_path = 'E:/NC/FTD-Data.csv'
data = pd.read_csv(data_path, encoding='gbk')

# Extract numerical features and target variables
X_numeric = data.iloc[:, :18].values.astype(np.float32)
y = data.iloc[:, -2:].values.astype(np.float32)
concrete_types = data['CT'].values  # For stratification

# Extract and preprocess text features (columns 18 to 21)
X_text = data.iloc[:, 18:22].astype(str)
X_text = X_text.applymap(preprocess_text)

# Tokenize using NLTK's word_tokenize
tokenized_texts = X_text.applymap(word_tokenize)

# Create vocabulary and word index (0 reserved for padding)
vocab = set([word for sublist in tokenized_texts.values.flatten() for word in sublist])
word_index = {word: i + 1 for i, word in enumerate(vocab)}
vocab_size = len(word_index)
print("Vocabulary size:", vocab_size)

# Save the vocabulary index as CSV and JSON
vocab_df = pd.DataFrame(list(word_index.items()), columns=['Word', 'Index'])
vocab_df.to_csv(os.path.join(output_dir, 'vocab_index.csv'), index=False)
with open(os.path.join(output_dir, 'vocab_index.json'), 'w', encoding='utf-8') as json_file:
    json.dump(word_index, json_file, ensure_ascii=False, indent=4)

# Convert tokenized texts to sequences
sequences = tokenized_texts.applymap(lambda x: [word_index[word] for word in x])

# Determine maximum lengths for each text column and save them
max_lengths = [max(len(seq) for seq in sequences.iloc[:, i]) for i in range(4)]
max_lengths_df = pd.DataFrame({'Column': X_text.columns, 'Max_Length': max_lengths})
max_lengths_df.to_csv(os.path.join(output_dir, 'max_lengths.csv'), index=False)
print("Max lengths of each text column:", max_lengths)

# ----------------------------------------
# Stratified shuffle split for train and test sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_numeric, concrete_types):
    X_train_numeric, X_test_numeric = X_numeric[train_index], X_numeric[test_index]
    X_train_text, X_test_text = X_text.iloc[train_index], X_text.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    concrete_types_train, concrete_types_test = concrete_types[train_index], concrete_types[test_index]

# Convert texts to sequences and pad them for training and testing
X_train_text_padded = []
X_test_text_padded = []
for col in X_text.columns:
    train_texts = [word_tokenize(text) for text in X_train_text[col]]
    test_texts = [word_tokenize(text) for text in X_test_text[col]]
    train_sequences = [[word_index[word] for word in text if word in word_index] for text in train_texts]
    test_sequences = [[word_index[word] for word in text if word in word_index] for text in test_texts]
    max_len = max_lengths[X_text.columns.get_loc(col)]
    X_train_text_padded.append(pad_sequences(train_sequences, maxlen=max_len, padding='post'))
    X_test_text_padded.append(pad_sequences(test_sequences, maxlen=max_len, padding='post'))

# Standardize the numerical features and save the scaler
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)
scaler_path = os.path.join(output_dir, 'scaler.pkl')
joblib.dump(scaler, scaler_path)

# Print the shape of each padded text feature
for column_name, padded_array in zip(X_text.columns, X_train_text_padded):
    print(f"Shape of {column_name}: {padded_array.shape}")

# ----------------------------------------
# Define the model creation function with GRU layers
def create_model(max_lengths, vocab_size, embedding_dim):
    inputs = []
    last_outputs = []
    sequence_outputs = []
    for i, max_len in enumerate(max_lengths):
        # Text input layer
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(
            input_dim=vocab_size + 1,
            output_dim=embedding_dim,
            input_length=max_len,
            trainable=True,
            mask_zero=True,
            name=f"embedding_{i+1}"
        )(input_layer)
        # Bidirectional GRU with return_sequences and return_state
        gru_layer = Bidirectional(
            GRU(16, return_sequences=True, return_state=True),
            name=f"bigru_{i+1}"
        )
        # For Bidirectional GRU, outputs: [sequence_output, forward_h, backward_h]
        gru_outputs = gru_layer(embedding_layer)
        sequence_output = gru_outputs[0]
        forward_h = gru_outputs[1]
        backward_h = gru_outputs[2]
        # Concatenate the final hidden states
        final_state = Concatenate(name=f"final_state_{i+1}")([forward_h, backward_h])
        inputs.append(input_layer)
        last_outputs.append(final_state)
        sequence_outputs.append(sequence_output)  # For analysis (ignored in loss)
    # Numerical input
    input_numerical = Input(shape=(18,), dtype='float32', name="numerical_input")
    inputs.append(input_numerical)
    # Combine text and numerical features
    combined_features = Concatenate(name="combined_features")(last_outputs + [input_numerical])
    # Fully connected layers
    x = Dense(128, activation='relu', kernel_initializer=HeNormal(), name="dense_1")(combined_features)
    x = Dropout(0.2, name="dropout_1")(x)
    x = Dense(64, activation='relu', kernel_initializer=HeNormal(), name="dense_2")(x)
    x = Dropout(0.2, name="dropout_2")(x)
    x = Dense(32, activation='relu', kernel_initializer=HeNormal(), name="dense_3")(x)
    x = Dropout(0.2, name="dropout_3")(x)
    final_outputs = Dense(2, name="final_output")(x)
    # Create model: output includes final prediction and sequence outputs (ignored in loss)
    model = Model(inputs=inputs, outputs=[final_outputs] + sequence_outputs, name="analysis_model")
    # Compile model: compute loss only on 'final_output'
    loss_dict = {'final_output': 'mae'}
    loss_weights = {'final_output': 1.0}
    for i in range(len(sequence_outputs)):
        loss_dict[f"bigru_{i+1}"] = None
        loss_weights[f"bigru_{i+1}"] = 0.0
    model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_dict, loss_weights=loss_weights)
    return model

# ----------------------------------------
# Initialize metrics and training history storage
metrics_dict = {
    'MAE': {0: [], 1: []},
    'MSE': {0: [], 1: []},
    'RMSE': {0: [], 1: []},
    'R2': {0: [], 1: []}
}
loss_history = []

# ----------------------------------------
# 5-fold stratified cross-validation training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_counter = 0
for train_idx, val_idx in skf.split(X_train_numeric_scaled, concrete_types_train):
    fold_counter += 1
    print(f"Training fold {fold_counter}/5")
    # Split numeric data and targets for the current fold
    X_train_fold_numeric = X_train_numeric_scaled[train_idx]
    X_val_fold_numeric = X_train_numeric_scaled[val_idx]
    y_train_fold = y_train[train_idx]
    y_val_fold = y_train[val_idx]
    
    # Create a new model
    model = create_model(max_lengths, vocab_size, embedding_dim=64)
    
    # Prepare text inputs for the current fold
    train_text_inputs = [X_train_text_padded[i][train_idx] for i in range(len(max_lengths))]
    val_text_inputs = [X_train_text_padded[i][val_idx] for i in range(len(max_lengths))]
    # Append numeric data as the last input
    train_inputs = train_text_inputs + [X_train_fold_numeric]
    val_inputs = val_text_inputs + [X_val_fold_numeric]
    
    # Train the model
    history = model.fit(
        train_inputs, y_train_fold,
        validation_data=(val_inputs, y_val_fold),
        epochs=3, batch_size=64, verbose=1
    )
    # Record training history
    loss_history.append(history.history)
    # Predict on validation set for metric evaluation
    y_pred = model.predict(val_inputs)
    final_predictions = y_pred[0]  # Final prediction output
    
    # Evaluate the model on the validation set for each target
    for target in range(2):
        mae_value = mean_absolute_error(y_val_fold[:, target], final_predictions[:, target])
        mse_value = mean_squared_error(y_val_fold[:, target], final_predictions[:, target])
        rmse_value = np.sqrt(mse_value)
        r2_value = r2_score(y_val_fold[:, target], final_predictions[:, target])
        metrics_dict['MAE'][target].append(mae_value)
        metrics_dict['MSE'][target].append(mse_value)
        metrics_dict['RMSE'][target].append(rmse_value)
        metrics_dict['R2'][target].append(r2_value)
    # Save the model for the current fold
    model_save_path = os.path.join(output_dir, f'model_fold_{fold_counter}.h5')
    model.save(model_save_path)
    print(f"Fold {fold_counter} model saved to: {model_save_path}")

# Save the training history of each fold to CSV files
for i, history in enumerate(loss_history):
    df_history = pd.DataFrame(history)
    history_csv_path = os.path.join(output_dir, f'fold_{i+1}_history.csv')
    df_history.to_csv(history_csv_path, index=False)
    print(f"Fold {i+1} training history saved to: {history_csv_path}")

# Plot and save loss curves (training and validation) for each fold
for i, history in enumerate(loss_history):
    plt.figure(figsize=(8, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves for Fold {i+1}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    loss_plot_path = os.path.join(output_dir, f'Loss_Curve_Fold_{i+1}.svg')
    plt.savefig(loss_plot_path)
    plt.close()
    print(f"Fold {i+1} loss curve saved to: {loss_plot_path}")

# ----------------------------------------
# Test Phase: Evaluate each fold's model on the test set and store metrics

# Initialize a dictionary to store test metrics for each fold and target
test_metrics = {
    'Fold': [],
    'Target': [],
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R2': []
}

num_folds = 5
for fold in range(1, num_folds + 1):
    print(f"Processing test metrics for fold {fold}")
    # Load the model from the current fold
    model = create_model(max_lengths, vocab_size, embedding_dim=64)
    model.load_weights(os.path.join(output_dir, f'model_fold_{fold}.h5'))
    
    # Prepare test inputs: text inputs + numeric input
    test_text_inputs = [X_test_text_padded[i] for i in range(len(max_lengths))]
    test_inputs = test_text_inputs + [X_test_numeric_scaled]
    
    predictions = model.predict(test_inputs)
    pred_output = predictions[0] if isinstance(predictions, list) else predictions
    
    # Calculate metrics for each target on the test set
    for target in range(2):
        mae = mean_absolute_error(y_test[:, target], pred_output[:, target])
        mse = mean_squared_error(y_test[:, target], pred_output[:, target])
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test[:, target], pred_output[:, target])
        test_metrics['Fold'].append(f"Fold {fold}")
        test_metrics['Target'].append(f"target{target+1}")
        test_metrics['MAE'].append(mae)
        test_metrics['MSE'].append(mse)
        test_metrics['RMSE'].append(rmse)
        test_metrics['R2'].append(r2)
        print(f"Fold {fold} - Target {target+1} Metrics:")
        print(f"  MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

# Save detailed test metrics to CSV
df_test_metrics = pd.DataFrame(test_metrics)
metrics_csv_path = os.path.join(output_dir, 'Test_Metrics.csv')
df_test_metrics.to_csv(metrics_csv_path, index=False)
print(f"All test metrics saved to: {metrics_csv_path}")

# Calculate average metrics for each target across folds and save
avg_metrics = df_test_metrics.groupby('Target', as_index=False).mean(numeric_only=True)
avg_csv_path = os.path.join(output_dir, 'Average_Test_Metrics.csv')
avg_metrics.to_csv(avg_csv_path, index=False)
print("Average test metrics:")
print(avg_metrics)
print(f"Average test metrics saved to: {avg_csv_path}")

print("Training histories and test metrics for 5 folds have been saved.")
